In [82]:
import os
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
import time

In [83]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [84]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["question", "context"])

In [85]:
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [86]:
loader = DirectoryLoader('data/', glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

vector_store = Chroma.from_documents(texts, embeddings, collection_metadata={"hnsw:space": "cosine"}, persist_directory="stores/insurance_cosine")

load_vector_store = Chroma(persist_directory="stores/insurance_cosine", embedding_function=embeddings)

print("Vector DB Successfully Created!")

100%|██████████| 1/1 [00:00<00:00, 15.03it/s]


Vector DB Successfully Created!


In [87]:
retriever = load_vector_store.as_retriever(search_kwargs={"k":1})

In [88]:
repo_id = "HuggingFaceH4/zephyr-7b-alpha"

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.3, "max_length": 500}
)

c:\Users\prath\anaconda3\envs\ransh\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [89]:
chain_type_kwargs = {"prompt": prompt}

def qa_chain():
    qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
    )
    return qa

qa = qa_chain()

In [90]:
text_query = "what did tommy find in the attic?"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
Tommy found a book about school in the attic.
0.2968108654022217


In [91]:
text_query = "Describe the pages of the book"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
The text doesn't provide a description of the pages of the book. The conversation only mentions that Tommy found a book in his attic about school, and Margie asks what it's about. Margie then expresses her dislike for school, but there is no further information about the book's pages or content.
0.2934699058532715


In [92]:
text_query = "Which class did Margie had to attend?"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
Margie attended a school with a mechanical teacher.
0.29724931716918945


In [93]:
text_query = "Who was tommy?"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
Tommy was mentioned in the context as someone whom Margie addressed a question to. However, no further information about Tommy was provided in the given text to answer the question "Who was Tommy?".
0.29314661026000977


In [94]:
text_query = "Who was Margie?"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
Margie is not mentioned in the given context. The passage is describing how schools used to be in the past, and how Isaac Asimov was a renowned science-fiction author. Margie's identity or role in the passage is not clear.
0.29085373878479004


In [95]:
text_query = "how were teachers like earlier"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.

According to your mother's perspective, teachers were not taught to adjust to fit the mind of each student they teach. They were not taught to teach differently for each student based on their age or individual needs. This is a significant difference from how teachers are taught to teach today.
0.2929189205169678


In [96]:
text_query = "Where does tommy show superiority over margie"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
Tommy shows superiority over Margie when he nonchalantly says "Maybe" to her offer to read the book with him after school. This implies that he may not want to read with her, or he may have other plans. Margie, on the other hand, seems eager to continue reading the book.
0.2911202907562256


In [97]:
text_query = "give facial characterstics of the inspector"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
The inspector had a round little face with a red hue.
0.30387258529663086


In [100]:
text_query = "Why was Margie hurt"

start_time = time.time()
text_response = qa(text_query)
print(text_response['result'])
end_time = time.time()
print(end_time - start_time)



> Entering new RetrievalQA chain...

> Finished chain.
Margie was disappointed because she had been hoping they would take the mechanical teacher away altogether. She was hoping for a different kind of school, not the one she had.
1.829171895980835
